In [1]:

import argparse
import os

#set visible cuda devices
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from transformers import (AutoModelForSequenceClassification,
                          AutoModelForTokenClassification, 
                          AutoModelForCausalLM,
                          AutoModelForMaskedLM,
                          AutoModel,
)
import yaml
from tqdm import tqdm
import numpy as np

import sys
sys.path.append("../")

from data_utils.model_utils import count_trainable_parameters, freeze_model, unfreeze_model

In [2]:
# turn above into a function that accepts multiple models and returns the gpu memory needed
# it should take in a list of model names and return a dictionary of model names and gpu memory needed

def get_gpu_memory_needed(model_names):
    device = torch.device('cuda:0') 
    gpu_memory_needed = {}
    for model_name in tqdm(model_names):
        model = AutoModel.from_pretrained(model_name,
                                  )
        model.to(device)
        gpu_memory_needed[model_name] = torch.cuda.memory_allocated(device.index)/1024**3
    return gpu_memory_needed

In [3]:
model_names = [
    "nlpie/bio-mobilebert",
                    "nlpie/tiny-biobert",
                    "roberta-base",
                    "nlpie/distil-biobert",
                    "dmis-lab/biobert-v1.1",
                     "meta-llama/Llama-2-7b-hf"
                     ]

gpu_memory_needed = get_gpu_memory_needed(model_names)

 17%|█▋        | 1/6 [00:01<00:09,  1.96s/it]Some weights of BertModel were not initialized from the model checkpoint at nlpie/tiny-biobert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 2/6 [00:02<00:03,  1.09it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 50%|█████     | 3/6 [00:02<00:01,  1.53it/s]Some weights of BertModel were not initialized from the model checkpoint at nlpie/distil-biobert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:11<00:00,  1.84s/it]


In [6]:
gpu_memory_needed 

{'nlpie/bio-mobilebert': 0.09157848358154297,
 'nlpie/tiny-biobert': 0.05176830291748047,
 'roberta-base': 0.46604251861572266,
 'nlpie/distil-biobert': 0.24604511260986328,
 'dmis-lab/biobert-v1.1': 0.40447139739990234,
 'meta-llama/Llama-2-7b-hf': 24.739288330078125}

In [7]:
# save to json
import json
with open('../gpu_memory_needed.json', 'w') as fp:
    json.dump(gpu_memory_needed, fp)